In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 多クラス分類
前章では２つのグループに分けることを考えたが、この章では３つ以上のグループに分けることを考える。

### ラベル推定式
$d$次元の特徴ベクトル$\boldsymbol{x}$が与えられているとき、分類の候補になる$K$個のクラスの集合$\mathcal{Y}=\{\mathcal{C}_{1}, \mathcal{C}_{2}, \cdots , \mathcal{C}_{K}\}$   wokangaeru 

線形多クラス分類では各カテゴリを$\hat{y} \in \mathcal{Y}$に重みベクトル$\boldsymbol{w}_{y} \in \mathbb R ^ d$を用意し、事例$\boldsymbol{x} \in \mathbb R ^ d$と重みベクトル$\boldsymbol{w}_{y}$との内積を計算し、最も高い内積値が計算されたカテゴリ$y \in \mathcal{Y}$に分類する。
よって多クラス分類のラベル推定式は以下のようになる。
$$
\hat{y} = \argmax_{y \in \mathcal{Y}}\boldsymbol{w}_y^\top\boldsymbol{x}
$$

### 多クラスロジスティック回帰
線形多クラス分類を実現するモデルの一つで事例$\boldsymbol{x}$をクラス$\mathcal{C}$に分類する条件付確率$P(\hat{y}=\mathcal{C}_j | \boldsymbol{x})$を以下のように求める。
$$
P(\hat{y}=\mathcal{C}_j | \boldsymbol{x}) = \frac{e^{\boldsymbol{w}_j^\top\boldsymbol{x}}}{\displaystyle \sum^{K}_{k=1}e^{\boldsymbol{w}_k^\top\boldsymbol{x}}}
$$
$a_j = \boldsymbol{w}_j^\top \boldsymbol{x}$と置くと上式は
$$
P(\hat{y}=\mathcal{C}_j | \boldsymbol{x}) = \frac{e^{a_j}}{\displaystyle \sum^{K}_{k=1}e^{a_k}}
$$
となり、この式はソフトマックス関数の形をしているため$\sigma$をソフトマックス関数とすると以下のように置く。
$$
\sigma(\boldsymbol{a})_j = P(\hat{y}=\mathcal{C}_j|\boldsymbol{x}) = \frac{e^{a_j}}{\displaystyle \sum^{K}_{k=1}e^{a_k}} 
$$
ここで、$\boldsymbol{a}$は
$$
\boldsymbol{a} = \begin{pmatrix}
\boldsymbol{w}_1^\top \boldsymbol{x} \\ \boldsymbol{w}_2^\top \boldsymbol{x} \\ \boldsymbol{w}_3^\top \boldsymbol{x} \\ \vdots \\ \boldsymbol{w}_K^\top \boldsymbol{x} 
\end{pmatrix}
$$
であり、$\sigma(\boldsymbol{a})_j$は$\sigma(\boldsymbol{a})$の出力の$j$番目の要素である。


In [10]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

x = np.array([-5, 2, 1000])
y = softmax(x)

print(f'出力: {y}')
print(f'出力ベクトルの大きさ: {np.sum(y)}')

出力: [ 0.  0. nan]
出力ベクトルの大きさ: nan


C:\Users\eu21052\AppData\Local\Temp\ipykernel_10332\2871618204.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / np.sum(np.exp(x))
C:\Users\eu21052\AppData\Local\Temp\ipykernel_10332\2871618204.py:2: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / np.sum(np.exp(x))


以上の結果から分かるようにソフトマックス関数はシグモイド関数同様に出力を確率とみなすことができる。ただしシグモイド関数と異なる点は、ソフトマックス関数はクラス数を２以上に設定できる。したがってソフトマックス関数はシグモイド関数を拡張したものとみることができる。したがって２値分類ではシグモイド関数を、多クラス分類ではソフトマックス関数を使用する。
しかし上で実装したコードでは以下のようなことが起こってしまう。

In [12]:
x = np.array([-5, 2, 1000])
y = softmax(x)

print(f'出力: {y}')
print(f'出力ベクトルの大きさ: {np.sum(y)}')

出力: [ 0.  0. nan]
出力ベクトルの大きさ: nan


C:\Users\eu21052\AppData\Local\Temp\ipykernel_10332\2871618204.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / np.sum(np.exp(x))
C:\Users\eu21052\AppData\Local\Temp\ipykernel_10332\2871618204.py:2: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / np.sum(np.exp(x))


上記のコードは入力ベクトルの要素の１つの値を1000に変えソフトマックス関数に入力している。しかし出力結果を見てもわかるようにソフトマックス関数の出力がオーバーフローしてしまっている。そこで今後使用するソフトマックス関数を以下のように変更する。
$$
\sigma(\boldsymbol{a})_j = \frac{e^{(a_j - b)}}{\displaystyle \sum^{K}_{k=1}e^{(a_k - b)}}
$$
ここで$b$は任意の実数である。この$b$を適切に設定することでオーバーフローを防ぐ。
またこの式は、
$$
\begin{align}
\sigma(\boldsymbol{a})_j &= \frac{e^{a_j}}{\displaystyle \sum^{K}_{k=1}e^{a_k}} \notag \\
&= \frac{e^{(a_j - b + b)}}{\displaystyle \sum^{K}_{k=1}e^{(a_k - b + b)}} \notag \\
&= \frac{e^be^{(a_j - b)}}{\displaystyle e^b\sum^{K}_{k=1}e^{(a_k - b)}} \notag \\
&= \frac{e^{(a_j - b)}}{\displaystyle \sum^{K}_{k=1}e^{(a_k - b)}} \notag 
\end{align}
$$
となるので変更後の式でも問題ないことがわかる。

### 最尤推定
2値分類同様最尤推定を行っていくが大まかな流れは2値分類と同じである。学習事例$(\boldsymbol{x}, \boldsymbol{y})$に対するモデルパラメータ$\boldsymbol{W}$の尤度$\hat{l}_{\boldsymbol{x},\boldsymbol{y}}$は、
$$
\hat{l}_{\boldsymbol{x},\boldsymbol{y}} = P(\hat{y} = \mathcal{C}_j|\boldsymbol{x})
$$
多クラスロジスティック回帰の条件付確率は$\boldsymbol{p}=\sigma(\boldsymbol{W}\boldsymbol{x})$なので、
$$
\hat{l}_{\boldsymbol{x},\boldsymbol{y}} = p_j = \prod^{K}_{k=1}\left\{
    \begin{array}{l}
        p_k \quad (y_k = 1) \\
        1 \quad (y_k = 0)
    \end{array}
    \right . 
    = \prod^{K}_{k=1}p_k^{y_k}
$$
またデータ$D$全体の尤度$\hat{L}_D$は、
$$
\hat{L}_D(\boldsymbol{W}) = \prod^{N}_{i=1}\hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W})
$$
これを目的関数とし、最大化させるような$\boldsymbol{W}$を求める。また2値分類同様に目的関数を以下のように変更する。
$$
\begin{align}
\mathcal{\hat{L}}_D(\boldsymbol{W}) &= -log\hat{L}_D(\boldsymbol{W}) \notag \\
&= -\sum^{N}_{i=i} log \hat{l}_{\boldsymbol{x}_i, \boldsymbol{y}_i}(\boldsymbol{W}) \notag
\end{align}
$$


### 確率的勾配降下法
2値分類と同様に確率的勾配降下法によってパラメータを推定していく。
$\boldsymbol{W}$の更新式は
$$
\boldsymbol{W}^{(t+1)} = \boldsymbol{W}^{(t)} + \eta_t \nabla log\hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W}^{(t)})
$$
$\boldsymbol{W}$に対しての偏微分はわかりづらいので$t$回目の反復で$\boldsymbol{W}^{(t)}$の列ベクトル$\boldsymbol{w}_j^{(t)}$毎に偏微分を計算しすべての$j$に対して重みベクトル$\boldsymbol{w}_j^{(t)}$を更新する式に変更する。なお$j$は$\boldsymbol{W}$の列番号である($1 \leqq j \leqq K$)。
$$
\left .
\boldsymbol{w}_j^{(t+1)} = \boldsymbol{w}_j^{(t)} + \eta_t \frac{\partial log \hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W})}{\partial \boldsymbol{w}_j} \right|_{\boldsymbol{W}=\boldsymbol{W}^{(t)}}
$$
ここで$\frac{\partial log \hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W})}{\partial \boldsymbol{w}_j}$について考える。
まず、
$$
log \hat{l}_{\boldsymbol{x}_i, \boldsymbol{y}_i} = log \prod^{K}_{k=1}p_k^{y_k} = \sum^{K}_{k=1} y_k log p_k 
$$
なので
$$
\begin{align}
\frac{\partial log \hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W})}{\partial \boldsymbol{w}_i}
&= \frac{\partial}{\partial \boldsymbol{w}_i}\sum^{K}_{k=1} \notag \\
&= \sum^{K}_{k=1}\frac{y_k}{p_k} \frac{\partial p_k}{\partial \boldsymbol{w}_i} \notag \\
\end{align}
$$
また、
$$
\frac{\partial p_k}{\partial \boldsymbol{w}_i} = \frac{\partial}{\partial \boldsymbol{w}_i}\sigma(a)_k =\frac{\partial p_k}{\partial a_i} \frac{\partial a_i}{\partial \boldsymbol{w}_i}
$$
ここで  
(i)$k = i$のとき  
$S = \sum\limits^{K}_{k=1}e^{x_k}$とおくと$p_k = \frac{e^{x_k}}{S}$より
$$
\begin{align}
\frac{\partial p_k}{\partial a_i} &= \frac{e^{x_i}S - e^{x_i}e^{x_i}}{S^2} \notag \\
&= \frac{e^{x_i}}{S} - (\frac{e^{x_i}}{S}) ^ 2 \notag \\
&= y_i - y_i ^ 2 \notag \\
&= y_i(1 - y_i) \notag 
\end{align}
$$
(ii)$k = i$のとき
(i)と同様に
$$
\begin{align}
\frac{\partial p_k}{\partial a_i} &= - \frac{e^{x_k} e^{x_i}}{S ^ 2} \notag \\
&= - \frac{e^{x_k}}{S} \frac{e^{x_i}}{S} \notag \\
&= - y_k y_i \notag
\end{align}
$$
(i)(ii)より、
$$
\frac{\partial p_k}{\partial a_i} 
= \left \{ \begin{array}{l}
    y_k(1 - y_k) \;\;\; (k = i)\\
    -y_k y_i \;\;\;\;\;\;\;\;\;\; (k \neq i)
\end{array} \right .
$$
また、
$$
\delta_{ik} = \left \{ \begin{array}{l}
    1 \;\;\; (k = i) \\
    0 \;\;\; (k \neq i)
\end{array} \right .
$$
を使用すると、
$$
\frac{\partial p_k}{\partial a_i} = y_k(\delta_{ik} - y_i)
$$
となるので、
$$
\frac{\partial p_k}{\partial w_i} = y_k(\delta_{ik} - y_i) \boldsymbol{x}
$$
以上より、
$$
\begin{align}
\frac{\partial log \hat{l}_{\boldsymbol{x}, \boldsymbol{y}}(\boldsymbol{W})}{\partial \boldsymbol{w}_j}
&= \sum^{K}_{k=1} \frac{y_k}{p_k} p_k(\delta_{ik} - p_i)\boldsymbol{x} \notag \\
&= \sum^{K}_{k=1} y_k(\delta_{ik} - p_i) \boldsymbol{x} \notag \\
&= (y_i - p_i \sum^{K}_{k=1} p_k) \boldsymbol{x} \notag \\
&= (y_i - p_i) \boldsymbol{x} \notag 
\end{align}
$$
したがって確率的勾配降下法による多クラスロジスティック回帰のパラメータ更新式は、
$$
\boldsymbol{w}_j^{(t+1)} = \boldsymbol{w}_j^{(t)} + \eta_t(y_j - p_j^{(t)})\boldsymbol{x}
$$


### 実装
多クラスロジスティック回帰を確率的勾配降下法にて実装する

In [ ]:
import gzip
import sys
import struct
import urllib.request
from pathlib import Path
import numpy as np

def read_image(fi):
    magic, n, rows, columns = struct.unpack(">IIII", fi.read(16))
    assert magic == 0x00000803
    assert rows == 28
    assert columns == 28
    rawbuffer = fi.read()
    assert len(rawbuffer) == n * rows * columns
    rawdata = np.frombuffer(rawbuffer, dtype='>u1', count=n*rows*columns)
    return rawdata.reshape(n, rows, columns).astype(np.float32) / 255.0

def read_label(fi):
    magic, n = struct.unpack(">II", fi.read(8))
    assert magic == 0x00000801
    rawbuffer = fi.read()
    assert len(rawbuffer) == n
    return np.frombuffer(rawbuffer, dtype='>u1', count=n)

def openurl_gzip(url):
    request = urllib.request.Request(
        url,
        headers={
            "Accept-Encoding": "gzip",
            "User-Agent": "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11", 
        })
    response = urllib.request.urlopen(request)
    return gzip.GzipFile(fileobj=response, mode='rb')

def save_mnist():
    if Path("data/mnist.npz").exists():
        return
    np.savez_compressed(
        "data/mnist",
        train_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")),
        train_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")),
        test_x=read_image(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")),
        test_y=read_label(openurl_gzip("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"))
    )

save_mnist()
data = np.load("data/mnist.npz")
print("Training data (X):", data["train_x"].shape, data["train_x"].dtype)
print("Training data (Y):", data["train_y"].shape, data["train_y"].dtype)
print("Test data (X):", data["test_x"].shape, data["test_x"].dtype)
print("Test data (Y):", data["test_y"].shape, data["test_y"].dtype)

In [ ]:
class LogisticClassifier():
    
    def __init__(self):
        self.W = None

    def softmax(self, a):
        # refer 6.5.1
        ea = np.exp(a - np.max(a))
        return ea / np.sum(ea)

    def train(self, X, Y, num_class, eta=1e-3, alpha=1e-6, epoch=100000, eps=1e-6):
        N = X.shape[0]
        self.W = np.random.uniform(size=(X.shape[-1], num_class))
        for t in range(epoch):
            i = np.random.choice(N)
            hat_y = self.predict_proba(X[i])
            # to one-hot vector
            y = np.zeros(num_class)
            y[Y[i]] = 1.0
            delta = (y - hat_y) * X[i].reshape((-1, 1)) - 2 * alpha * self.W / N
            if np.sum(np.abs(delta)) < eps:
                break
            self.W += eta * delta
        return self

    def predict_proba(self, x):
        value = x @ self.W
        if len(value.shape) < 2:
            return self.softmax(value).flatten()
        else:
            return np.apply_along_axis(self.softmax, axis=1, arr=value)

    def predict(self, x):
        proba = self.predict_proba(x)
        return np.argmax(proba, axis=1)

In [ ]:
X_train = data["train_x"]


def to_feature(X):
    return np.c_[X.reshape((X.shape[0], -1)), np.ones(X.shape[0])]


X_train = to_feature(X_train)
model = LogisticClassifier().train(X=X_train, Y=data["train_y"], num_class=10)